In [ ]:
import pandas as pd
from load_ulg import load_ulg, timeframe
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial.transform import Rotation as R
import numpy as np

In [ ]:
output_topic = "actuator_motors_mux"
output_topic2 = "actuator_motors_mux"

### Loading the logfile into a Pandas dataframe

In [ ]:
ulog_files = [
    # "logs/2024-01-09-jonas-hitl/log_19_2024-1-8-21-53-38.ulg"
    # "logs/2024-01-09-jonas-hitl/log100.ulg"
    # "logs/2024-01-09-jonas-hitl/no_delay.ulg"
    # "logs/2024-01-09-jonas-hitl/15_14_37.ulg"
    # "logs/2024-01-12-jonas-hitl/log_70_2022-1-1-04-07-10.ulg"
    # "logs/2024-01-12-jonas-hitl/log_72_2022-1-1-04-16-32.ulg"
    # "logs/2024-01-12-jonas-hitl/log_73_2022-1-1-04-17-42.ulg",
    # "logs/2024-01-12-pavel policy test/log_76_2024-1-12-17-25-30.ulg",
    # "logs/2024-01-12-pavel policy test/log_77_2024-1-12-17-27-50.ulg"
    # "logs/2024-01-15-pavel/log_82_2024-1-15-12-34-06.ulg", # long activation
    # "logs/2024-01-15-pavel/log_83_2024-1-15-12-48-46.ulg"
    # "logs/2024-01-15-pavel/log_86_2024-1-15-15-39-52.ulg" # clean history
    # "logs/2024-01-15-jonas-sim/log_39_2024-1-15-12-45-08.ulg",
    # "logs/2024-01-15-jonas-sim/log_40_2024-1-15-12-45-08.ulg"
    # "logs/2024-01-15-jonas-sim/log_41_2024-1-15-13-12-30.ulg"
    # "logs/2024-01-16-pavel-succesful-flight/log_87_2024-1-16-14-49-56.ulg",
    # "logs/2024-01-17-pre-test/log_90_2024-1-15-13-04-20.ulg"
    # "logs/2024-01-17-pavel/log_93_2024-1-17-15-11-38.ulg"
    "logs/2024-01-17-pavel/log_94_2024-1-17-15-14-04.ulg" # trajectory tracking 1x
]
dfs = [load_ulg(file) for file in ulog_files]

### Exploring the datapoints in the dataframe

In [ ]:
grep = "local"
[col for col in dfs[0].columns if grep in col]

### Looking at the outputs to find the relevant timeframe

In [ ]:
for i, (file, df) in enumerate(zip(ulog_files, dfs)):
    # relevant_df = df[[*[f"{output_topic}_control[{i}]" for i in range(4)], "rl_tools_multiplexer_status_active"]].dropna(how="all")
    plt.figure()
    plt.title(f"file {i}: {file}")
    handles = []
    labels = []
    for i in [3]:
        s = df[f"{output_topic}_control[{i}]"].dropna()
        plt.plot(s.index, s, label=f"{output_topic} motor {i}")
        s2 = df[f"{output_topic2}_control[{i}]"].dropna()
        plt.plot(s2.index, s2, label=f"{output_topic2} motor {i}")
    plt.xlabel("Time [s]")
    plt.ylabel("Throttle [0-1]")
    plt.legend()
    plt.twinx()
    plt.ylabel("active [true/false]")
    if "rl_tools_multiplexer_status_active" in df.columns:
        s = df["rl_tools_multiplexer_status_active"].dropna()
        plt.plot(s.index, s, label="active", color="red")
    plt.legend()
    intervals = df["actuator_motors_control[0]"].index.to_series().diff().dropna()
    interval_mean = intervals.mean()
    interval_std = intervals.std()
    print(f"file {i}: interval mean: {interval_mean}, std: {interval_std}")

    

In [ ]:
timeframes = [
    # (20, 80)
    # (35.42, 40.47),
    # (60, 80),
    # (27.75, 28.5),
    # (93.5, 94.5),
    # (0, 1000000000)
    # (20, 90)
    # (4.5, 10),
    # (63.9, 64.5),
    # (65, 80)
    # (63.9, 64.5),
    # (63.95, 64.1),
    # (1.8, 2.2),
    # (69, 70.5),
    # (76.2, 77.5),
    # (66, 68)
    # (56.5, 57.5)
    # (71, 72),
    # (86, 87),
    (31, 65),
    # (64.5, 65.5),
    # (45, 80),
]
dfs_tf = [timeframe(df, time_start, time_end) for df, (time_start, time_end) in zip(dfs, timeframes)]

In [ ]:
df = dfs_tf[0]
fig, axs = plt.subplots(9, 1, figsize=(10, 20), sharex=True)
fig.suptitle(f"Policy Analysis {ulog_files[0]}")
curr = 0
sa = df["rl_tools_multiplexer_status_active"].dropna()
axs[curr].plot(sa.index, sa, label="active")
axs[curr].legend()
axs[curr].set_ylabel("active [true/false]")
twin = axs[curr].twinx()
s = df["rl_tools_policy_status_exit_reason"].dropna()
twin.plot(s.index, s, label="active", color="red")
twin.set_ylabel("exit reason")
curr += 1
axs[curr].set_title("Policy Input")
for dim, axis in zip(range(3), ["x", "y", "z"]):
    s = df[f"rl_tools_policy_input_state_observation[{dim}]"].dropna()
    axs[curr].plot(s.index, s, label=f"position {axis}")
axs[curr].legend()
curr += 1
# for dim, axis in zip(range(3), ["x", "y", "z"]):
#     s = df[f"vehicle_local_position_{axis}"].dropna()
#     axs[curr].plot(s.index, s, label=f"position (actual) {axis}")
# axs[curr].legend()
# curr += 1
# for dim, axis in zip(range(4), ["w", "x", "y", "z"]):
#     s = df[f"rl_tools_command_target_orientation[{dim}]"].dropna()
#     axs[curr].plot(s.index, s, label=f"target orientation {axis}")
# axs[curr].legend()
# curr += 1
# for dim, axis in zip(range(4), ["w", "x", "y", "z"]):
#     s = df[f"vehicle_attitude_q[{dim}]"].dropna()
#     axs[curr].plot(s.index, s, label=f"orientation {axis} (actual)")
# axs[curr].legend()
# curr += 1
sR = df[[*[f"rl_tools_policy_input_state_observation[{dim+3}]" for dim in range(9)]]].dropna()
qs = np.array([R.from_matrix(sR.values[i].reshape(3, 3)).as_quat() for i in range(len(sR))])
for dim, axis in zip(range(4), ["x", "y", "z", "w"]):
    axs[curr].plot(sR.index, qs[:, dim], label=f"orientation {axis}")
axs[curr].legend()
curr += 1
for dim, axis in zip(range(3), ["x", "y", "z"]):
    s = df[f"rl_tools_policy_input_state_observation[{dim+3+9}]"].dropna()
    axs[curr].plot(s.index, s, label=f"linear velocity {axis}")
    # s = df[f"vehicle_local_position_v{axis}"].dropna()
    # axs[curr].plot(s.index, s, label=f"linear velocity {axis} (actual)")

axs[curr].legend()
curr += 1
for dim, axis in zip(range(3), ["x", "y", "z"]):
    s = df[f"rl_tools_policy_input_state_observation[{dim+3+9+3}]"].dropna()
    axs[curr].plot(s.index, s, label=f"angular velocity {axis}")
    # s = df[f"vehicle_angular_velocity_xyz[{dim}]"].dropna()
    # axs[curr].plot(s.index, s, label=f"angular velocity {axis} (actual)")
axs[curr].legend()
curr += 1
for dim in range(4):
    s = df[f"actuator_motors_mux_control[{dim}]"].dropna()
    axs[curr].plot(s.index, s, label=f"motor {dim}")
    axs[curr].scatter(s.index, s)
axs[curr].legend(loc="center left")
for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)
curr += 1

s = df["vehicle_acceleration_xyz[2]"].dropna()
axs[curr].plot(s.index, s, label="acceleration z")
axs[curr].set_title("Z acceleration (FRD)")


plt.savefig("figures/analysis.pdf")
plt.show()

In [ ]:
policy_outputs = dfs_tf[0][[*[f"actuator_motors_rl_tools_control[{i}]" for i in range(4)]]].dropna()

In [ ]:
policy_outputs.index.to_series().diff().plot()

In [ ]:
from pyulog import ULog
ulog = ULog(ulog_files[0], None, False)

In [ ]:
names = [d.name for d in ulog.data_list]
actuator_motors = ulog.data_list[np.argmax([name == "actuator_motors_mux" for name in names])]

In [ ]:
any([np.isnan(v) for v in actuator_motors.data["control[0]"]])

In [ ]:
[m.message for m in ulog.logged_messages]

In [ ]:
df_tt = dfs[0]
start_time = df_tt["rl_tools_command_active"].idxmax()
end_time = (df_tt[df_tt.index > start_time]["rl_tools_command_active"] == False).idxmax()
print(f"start_time: {start_time} end_time: {end_time}")

df_tt_tf = df_tt[(df_tt.index > start_time) & (df_tt.index < end_time)]

trajectory_tracking_df = df_tt_tf[[*[f"rl_tools_command_target_position[{i}]" for i in range(3)], *[f"vehicle_local_position_{axis}" for axis in "xyz"]]].dropna(how="all")

target_position_df = trajectory_tracking_df[["rl_tools_command_target_position[0]", "rl_tools_command_target_position[1]"]].dropna()
plt.plot(target_position_df["rl_tools_command_target_position[0]"], -target_position_df["rl_tools_command_target_position[1]"])
local_position_df = trajectory_tracking_df[["vehicle_local_position_x", "vehicle_local_position_y"]].dropna()
plt.plot(local_position_df["vehicle_local_position_x"], -local_position_df["vehicle_local_position_y"])